In [3]:
import torch
import numpy as np
import nir
import json

In [20]:
graph = nir.read("braille_noDelay_bias_zero.nir")

In [21]:
graph.nodes.keys()

dict_keys(['fc1', 'fc2', 'input', 'lif1.lif', 'lif1.w_rec', 'lif2', 'output'])

In [22]:
graph.nodes["lif1.w_rec"].weight.shape

(38, 38)

In [6]:
def rework_braille_graph(graph):
    graph = nir.read(graph)
    hidden = np.array([graph.nodes["lif1.w_rec"].weight.shape[0]])
    subgraph = nir.NIRGraph(
        nodes = {"input": nir.Input(hidden), "lif1": graph.nodes["lif1.lif"], "lin": graph.nodes["lif1.w_rec"], "output": nir.Output(hidden)},
        edges = [("input", "lif"), ("lif", "lin"), ("lin", "lif"), ("lif", "output")]
    )
    new_graph = nir.NIRGraph(
        nodes = {"input": graph.nodes["input"], "fc1": graph.nodes["fc1"], "fc2": graph.nodes["fc2"], "lif1": subgraph, "lif2": graph.nodes["lif2"], "output": graph.nodes["output"]},
        edges = [("input", "fc1"), ("fc1", "lif1"), ("lif1", "fc2"), ("fc2", "lif2"), ("lif2", "output")]
    )
    return new_graph
new_zero = rework_braille_graph("braille_noDelay_bias_zero.nir")
nir.write("braille_noDelay_bias_zero_subgraph.nir", new_zero)
new_subtract = rework_braille_graph("braille_noDelay_noBias_subtract.nir")
nir.write("braille_noDelay_noBias_subtract_subgraph.nir", new_subtract)

In [19]:
import norse.torch.utils as nu
nu.from_nir(new_graph)(torch.empty(12))

Please read the Norse documentation for more information.
This warning can be turned off by setting from_nir(..., ignore_warnings=True)

Please read the Norse documentation for more information.
This warning can be turned off by setting from_nir(..., ignore_warnings=True)



(tensor([0., 0., 0., 0., 0., 0., 0.], grad_fn=<SuperSpikeBackward>),
 GraphExecutorState(state={'lif1': (RecurrentSequentialState(cache=tensor([-0.5638, -0.5395, -0.5250, -0.4674, -0.0355, -0.5286, -0.3670, -0.0750,
          0.0210, -0.0689, -0.0777, -0.2457, -0.1274,  0.0114, -1.0532, -0.3147,
         -0.4618, -0.9926, -0.6294,  0.0518,  0.0164, -0.8293,  0.0284,  0.0717,
         -0.5576, -0.1293, -0.0519, -0.7355, -0.3816, -0.0711, -0.0503, -0.0018,
         -0.4985, -0.0235, -0.8448,  0.0581, -0.0813, -0.1282, -0.4182, -0.2181,
         -0.1173, -0.3772, -0.7924, -0.8783, -0.0159, -0.4439, -0.0940, -0.5002,
         -0.6602, -0.9305, -0.6093, -0.0884, -0.0210, -0.0041, -0.1188],
        grad_fn=<ViewBackward0>), state=[LIFFeedForwardState(v=tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0

In [ ]:
nir.write("braille")

In [2]:
# m = torch.load("graph_Braille_snntorch_20231005_140408.pt", map_location="cpu")
m = torch.load("data/model_ref_20231017_092347.pt", map_location="cpu")
m

OrderedDict([('fc1.weight',
              tensor([[ 2.8084e-01,  2.4418e-01, -1.3830e-01,  1.5931e-01, -1.9656e-01,
                       -2.2425e-01, -1.4157e-01,  8.1548e-02, -7.4504e-02,  2.7401e-02,
                       -1.6093e-01,  4.6718e-01],
                      [ 7.8919e-02, -2.3811e-01, -1.2176e-01, -2.4662e-01,  1.9838e-02,
                       -1.8514e-01, -3.8402e-01, -1.3560e-01, -2.8875e-01, -1.2967e-01,
                       -4.7271e-01,  6.6722e-02],
                      [-3.3065e-01, -3.3431e-01, -6.6987e-02, -4.9142e-01, -7.6607e-02,
                       -4.0130e-01,  3.6282e-01, -1.5178e-01,  1.5726e-01,  1.1598e-01,
                       -1.9487e-01,  3.6942e-02],
                      [-2.2091e-01,  7.2034e-02, -7.7206e-02, -2.5981e-01, -8.3313e-02,
                       -3.0539e-01, -4.9439e-02,  1.5638e-01, -1.8072e-02, -3.8405e-01,
                        7.0956e-02, -1.0806e-01],
                      [ 3.8160e-02, -7.0475e-02, -1.1823e-01, -4.025

In [11]:
[f"{x:25s}: {m[x].shape}" for x in m.keys()]

['fc1.weight               : torch.Size([54, 12])',
 'fc1.bias                 : torch.Size([54])',
 'lif1.threshold           : torch.Size([])',
 'lif1.graded_spikes_factor: torch.Size([])',
 'lif1.reset_mechanism_val : torch.Size([])',
 'lif1.beta                : torch.Size([])',
 'lif1.alpha               : torch.Size([])',
 'lif1.recurrent.weight    : torch.Size([54, 54])',
 'lif1.recurrent.bias      : torch.Size([54])',
 'fc2.weight               : torch.Size([7, 54])',
 'fc2.bias                 : torch.Size([7])',
 'lif2.threshold           : torch.Size([])',
 'lif2.graded_spikes_factor: torch.Size([])',
 'lif2.reset_mechanism_val : torch.Size([])',
 'lif2.beta                : torch.Size([])',
 'lif2.alpha               : torch.Size([])']

In [12]:
m["lif1.recurrent.bias"]

tensor([-0.1245, -0.3106, -0.1095, -0.1270, -0.0118, -0.1970, -0.1959, -0.2759,
        -0.9350, -1.4093, -0.2976, -0.2276, -0.0083, -0.2961, -1.2493, -0.2905,
        -0.0033, -0.7371, -0.1016,  0.0106, -0.3685,  0.0207, -0.0099,  0.0553,
        -0.1271, -0.0452, -0.2831,  0.0017, -0.9921, -0.1141, -0.1855, -0.9965,
        -0.2120, -0.3354, -0.2935, -0.1500, -0.2232, -0.2993, -0.8319, -0.2552,
        -0.4994, -0.0224, -0.0825, -0.1633, -0.2028, -0.3138, -0.1279, -0.2158,
        -0.2923, -0.1653, -0.3423, -0.0666, -0.9605, -0.5594])

In [13]:
m["lif1.graded_spikes_factor"]

tensor(1.)

In [14]:
### OPTIMAL HYPERPARAMETERS
parameters_path = "data/parameters_ref_20231011.json"

with open(parameters_path) as f:
   parameters = json.load(f)

parameters

{'nb_hidden': 38,
 'alpha_r': 0.55,
 'alpha_out': 0.5,
 'beta_r': 0.9,
 'beta_out': 0.55,
 'lr': 0.005,
 'reg_l1': 0.0006,
 'reg_l2': 4e-06,
 'slope': 5}

In [15]:
to_array = lambda v, s: np.ones(s) * np.array(v)
n_inputs = 12
n_hidden = parameters["nb_hidden"]
n_out = 7
sub = nir.NIRGraph(
    nodes = {"input": nir.Input(np.array([n_inputs])),
             "lif": nir.CubaLIF(tau_syn=to_array(1 / (1 - m["lif1.beta"]), n_hidden), tau_mem=to_array(1 / (1 - m["lif1.alpha"]), n_hidden),
                                r=to_array(1, n_hidden), v_leak=to_array(0, n_hidden), v_threshold=to_array(m["lif1.threshold"], n_hidden),
                                w_in=to_array(1, n_hidden)),
             "lin": nir.Affine(weight=m["lif1.recurrent.weight"], bias=m["lif1.recurrent.bias"]),
             "output": nir.Output(np.array([n_hidden]))},
    edges = [("input", "lif"), ("lif", "lin"), ("lin", "lif"), ("lif", "output")]
)

In [16]:
g = nir.NIRGraph(
    nodes = {
        "input": nir.Input(np.array([n_inputs])),
        "fc1": nir.Affine(weight=m["fc1.weight"], bias=m["fc1.bias"]),
        "fc2": nir.Affine(weight=m["fc2.weight"], bias=m["fc2.bias"]),
        "lif2": nir.CubaLIF(tau_syn=to_array(1 / (1 - m["lif2.beta"]), n_out), tau_mem=to_array(1 / (1 - m["lif2.alpha"]), n_out),
                            r=to_array(1, n_out), v_leak=to_array(0, n_out), v_threshold=to_array(m["lif2.threshold"], n_out),
                            w_in=to_array(1, n_out)),
        "hidden": sub,
        "output": nir.Output(np.array([n_out]))
    },
    edges = [("input", "fc1"), ("fc1", "hidden"), ("hidden", "fc2"), ("fc2", "lif2"), ("lif2", "output")]
)

nir.write("braille_subgraph.nir", g)

In [17]:
g = nir.read("braille_subgraph.nir")
g

NIRGraph(nodes={'fc1': Affine(weight=array([[ 2.80835301e-01,  2.44179189e-01, -1.38303146e-01,
         1.59307152e-01, -1.96557730e-01, -2.24245831e-01,
        -1.41570821e-01,  8.15482959e-02, -7.45038241e-02,
         2.74006240e-02, -1.60926566e-01,  4.67182308e-01],
       [ 7.89186507e-02, -2.38108546e-01, -1.21755384e-01,
        -2.46622637e-01,  1.98381469e-02, -1.85139120e-01,
        -3.84024382e-01, -1.35601968e-01, -2.88749933e-01,
        -1.29673436e-01, -4.72705811e-01,  6.67222515e-02],
       [-3.30653995e-01, -3.34314525e-01, -6.69869110e-02,
        -4.91423279e-01, -7.66068026e-02, -4.01303530e-01,
         3.62817615e-01, -1.51778132e-01,  1.57258809e-01,
         1.15980618e-01, -1.94868326e-01,  3.69423330e-02],
       [-2.20907137e-01,  7.20338151e-02, -7.72058964e-02,
        -2.59807289e-01, -8.33134949e-02, -3.05385530e-01,
        -4.94394600e-02,  1.56375661e-01, -1.80724952e-02,
        -3.84050727e-01,  7.09564015e-02, -1.08058497e-01],
       [ 3.8160